<a href="https://colab.research.google.com/github/MeetSolanki100/Marathi_author_profiling/blob/main/version1_normal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [1]:
df = pd.read_csv('dataset1.csv', on_bad_lines='skip', delimiter=',')
print(df.head())

                   title                                 link  \
0  मेघे ढका तारा (भाग ४)  https://www.maayboli.com/node/86621   
1  मेघे ढका तारा (भाग ३)  https://www.maayboli.com/node/86619   
2  मेघे ढका तारा (भाग २)  https://www.maayboli.com/node/86617   
3  नसलेल्या मुलीची गोष्ट  https://www.maayboli.com/node/86616   
4  मेघे ढका तारा (भाग १)  https://www.maayboli.com/node/86615   

                             subtitle       author  \
0   लेखनाचा धागा Apr 13 2025 - 5:08am        Abuva   
1   लेखनाचा धागा Apr 13 2025 - 4:03am        Abuva   
2  लेखनाचा धागा Apr 12 2025 - 11:05pm        Abuva   
3   लेखनाचा धागा Apr 12 2025 - 1:26pm  अविनाश जोशी   
4  लेखनाचा धागा Apr 12 2025 - 11:39am        Abuva   

                                             content  
0  (भाग ३ -\nhttps://www.maayboli.com/node/86619\...  
1  (भाग २-\nhttps://www.maayboli.com/node/86617\n...  
2  (भाग १ -\nhttps://www.maayboli.com/node/86615\...  
3  नसलेल्या मुलीची गोष्ट\n18 जून 1997\nशिंदेवाडीत...  
4  "मैं ब

In [3]:
df.columns

Index(['title', 'link', 'subtitle', 'author', 'content'], dtype='object')

In [4]:
unique_authors = df['author'].nunique()
print(f"Total number of unique authors: {unique_authors}")

Total number of unique authors: 626


In [5]:
df['content'].nunique()

4934

In [6]:
df = df.drop_duplicates(subset=['title'], keep='first')
df = df.dropna()

In [7]:
df.shape

(4831, 5)

In [8]:
df['content'][0]

'(भाग ३ -\nhttps://www.maayboli.com/node/86619\n)\nप्रोजेक्ट थोडासा ओव्हरशूट झाला होता. पण डिलेज मॅनेजेबल होते. मिहिरला सूत्रं मिळाल्यावर त्यानं बरंच सावरून घेतलं होतं. एक प्रश्न उपस्थित झाला. डॉक्युमेंटेशन! पुण्याला परत पाठवणं शक्य नव्हतं, कारण इथे बरेच चेंजेस झाले होते. कमलनं चक्क नकार दिला. मिहिरला करणं अशक्य नसलं तरी त्याचं काम बघता, ते इतर कोणी करणं सोयिस्कर होतं. विचार करताना मिहिरसमोर नाव आलं - सरोज! पण त्या दिवसापासून सरोज त्याच्याशी जरा आढ्यतेनंच वागत होती. झटकून टाकत होती. विचारावं का तिला, मिहिर दुग्ध्यात पडला.\nत्यादिवशी कमलच्या एका प्रोसेसचं टेस्टिंग ठरलं. त्यामुळे त्याला उशीर होणार होता. तो मिहिरला म्हणाला, सरोजला आज सुपरमार्केटमधे जायचंय. मला उशीर होणार आहे. तूच घेऊन जा तिला. ही एक इष्टापत्तीच होती.\nमिहिरनं बेल वाजवली, तेंव्हा सरोजनंच दार उघडलं. त्यानं किल्ल्या पुढे केल्या. "कमल नाही आला?”\n"नाही, त्याला काम आहे.”\nती सटकलीच. तिचा आवाज चढला. "जरा बाहेर जाऊ म्हटलं तर याला काम! आम्ही घरात सडतोय, पण यांची कामं महत्त्वाची! मुलाकडे लक्ष नाही, बायकोकडे नाहीच, फक्त काम, काम आण

In [10]:
def preprocess_text(text):
    if isinstance(text, str):
        # Remove URLs
        text = re.sub(r'https?://\S+', '', text)
        # Remove HTML tags
        text = re.sub(r'<.*?>', '', text)
        # Remove extra whitespaces
        text = re.sub(r'\s+', ' ', text).strip()
    else:
        text = ''
    return text

In [11]:
df['clean_content'] = df['content'].apply(preprocess_text)

In [12]:
df['content'][0]


'(भाग ३ -\nhttps://www.maayboli.com/node/86619\n)\nप्रोजेक्ट थोडासा ओव्हरशूट झाला होता. पण डिलेज मॅनेजेबल होते. मिहिरला सूत्रं मिळाल्यावर त्यानं बरंच सावरून घेतलं होतं. एक प्रश्न उपस्थित झाला. डॉक्युमेंटेशन! पुण्याला परत पाठवणं शक्य नव्हतं, कारण इथे बरेच चेंजेस झाले होते. कमलनं चक्क नकार दिला. मिहिरला करणं अशक्य नसलं तरी त्याचं काम बघता, ते इतर कोणी करणं सोयिस्कर होतं. विचार करताना मिहिरसमोर नाव आलं - सरोज! पण त्या दिवसापासून सरोज त्याच्याशी जरा आढ्यतेनंच वागत होती. झटकून टाकत होती. विचारावं का तिला, मिहिर दुग्ध्यात पडला.\nत्यादिवशी कमलच्या एका प्रोसेसचं टेस्टिंग ठरलं. त्यामुळे त्याला उशीर होणार होता. तो मिहिरला म्हणाला, सरोजला आज सुपरमार्केटमधे जायचंय. मला उशीर होणार आहे. तूच घेऊन जा तिला. ही एक इष्टापत्तीच होती.\nमिहिरनं बेल वाजवली, तेंव्हा सरोजनंच दार उघडलं. त्यानं किल्ल्या पुढे केल्या. "कमल नाही आला?”\n"नाही, त्याला काम आहे.”\nती सटकलीच. तिचा आवाज चढला. "जरा बाहेर जाऊ म्हटलं तर याला काम! आम्ही घरात सडतोय, पण यांची कामं महत्त्वाची! मुलाकडे लक्ष नाही, बायकोकडे नाहीच, फक्त काम, काम आण

In [13]:
df['clean_content'][0]

'(भाग ३ - ) प्रोजेक्ट थोडासा ओव्हरशूट झाला होता. पण डिलेज मॅनेजेबल होते. मिहिरला सूत्रं मिळाल्यावर त्यानं बरंच सावरून घेतलं होतं. एक प्रश्न उपस्थित झाला. डॉक्युमेंटेशन! पुण्याला परत पाठवणं शक्य नव्हतं, कारण इथे बरेच चेंजेस झाले होते. कमलनं चक्क नकार दिला. मिहिरला करणं अशक्य नसलं तरी त्याचं काम बघता, ते इतर कोणी करणं सोयिस्कर होतं. विचार करताना मिहिरसमोर नाव आलं - सरोज! पण त्या दिवसापासून सरोज त्याच्याशी जरा आढ्यतेनंच वागत होती. झटकून टाकत होती. विचारावं का तिला, मिहिर दुग्ध्यात पडला. त्यादिवशी कमलच्या एका प्रोसेसचं टेस्टिंग ठरलं. त्यामुळे त्याला उशीर होणार होता. तो मिहिरला म्हणाला, सरोजला आज सुपरमार्केटमधे जायचंय. मला उशीर होणार आहे. तूच घेऊन जा तिला. ही एक इष्टापत्तीच होती. मिहिरनं बेल वाजवली, तेंव्हा सरोजनंच दार उघडलं. त्यानं किल्ल्या पुढे केल्या. "कमल नाही आला?” "नाही, त्याला काम आहे.” ती सटकलीच. तिचा आवाज चढला. "जरा बाहेर जाऊ म्हटलं तर याला काम! आम्ही घरात सडतोय, पण यांची कामं महत्त्वाची! मुलाकडे लक्ष नाही, बायकोकडे नाहीच, फक्त काम, काम आणि दारू. वीट आलाय या सगळ्याचा... तू काय तोंड

In [14]:

author_counts = df['author'].value_counts()
top_20_authors = author_counts.nlargest(20).index
top_authors_df = df[df['author'].isin(top_20_authors)]
top_authors_df


,title,link,subtitle,author,content,clean_content
8,मुंबईची मुलगी,https://www.maayboli.com/node/86582,लेखनाचा धागा Apr 11 2025 - 2:50am,निमिष_सोनार,"मुंबईतली रात्र नेहमीच गजबजलेली असते, पण काही व...","मुंबईतली रात्र नेहमीच गजबजलेली असते, पण काही व..."
10,छोटीशी मदत,https://www.maayboli.com/node/86577,लेखनाचा धागा Apr 9 2025 - 11:07pm,निमिष_सोनार,ब्राझीलच्या घनदाट अ‍ॅमेझॉन जंगलात एका लहानशा ग...,ब्राझीलच्या घनदाट अ‍ॅमेझॉन जंगलात एका लहानशा ग...
11,तुझ्यासाठी,https://www.maayboli.com/node/86581,लेखनाचा धागा Apr 4 2025 - 2:03pm,निमिष_सोनार,"पाऊस रिमझिम कोसळत होता. स्टेशनवर गर्दी होती, प...","पाऊस रिमझिम कोसळत होता. स्टेशनवर गर्दी होती, प..."
12,विश्वासाचा धागा,https://www.maayboli.com/node/86580,लेखनाचा धागा Apr 4 2025 - 1:31am,निमिष_सोनार,"एक मोठी कंपनी होती, जिथे राहुल नावाचा एक तरुण ...","एक मोठी कंपनी होती, जिथे राहुल नावाचा एक तरुण ..."
13,मोबदला,https://www.maayboli.com/node/86578,लेखनाचा धागा Apr 4 2025 - 12:32am,निमिष_सोनार,जुन्या काळातील युरोपच्या एका शांत गावात हेन्री...,जुन्या काळातील युरोपच्या एका शांत गावात हेन्री...
...,...,...,...,...,...,...
4922,चारचौघी - ४,https://www.maayboli.com/node/40191,लेखनाचा धागा Jan 16 2013 - 2:13am,बेफ़िकीर,"पंधरा दिवस झाले, पण निली आणि जया एकमेकींशी एक ...","पंधरा दिवस झाले, पण निली आणि जया एकमेकींशी एक ..."
4923,चारचौघी - ८,https://www.maayboli.com/node/40456,लेखनाचा धागा Jan 24 2013 - 11:21pm,बेफ़िकीर,नीलाक्षीने केंद्रामधून आज रजा काढलेली आहे हे त...,नीलाक्षीने केंद्रामधून आज रजा काढलेली आहे हे त...
4924,चारचौघी - २,https://www.maayboli.com/node/40135,लेखनाचा धागा Jun 12 2015 - 4:00am,बेफ़िकीर,पहाटे पाचपर्यंत पिझाचे अवशेष उरलेल्या डिशेस तश...,पहाटे पाचपर्यंत पिझाचे अवशेष उरलेल्या डिशेस तश...
4967,जातो जेव्हा तुला सोडुनिया दूर....,https://www.maayboli.com/node/38601,लेखनाचा धागा Oct 19 2012 - 9:13am,मी मधुरा,"जातो जेव्हा तुला सोडुनिया दूर,\nभटकते जीवन गाण...","जातो जेव्हा तुला सोडुनिया दूर, भटकते जीवन गाणे..."


In [15]:
top_20_authors

Index(['अज्ञातवासी', 'निमिष_सोनार', 'मिलिंद महांगडे', 'मिरिंडा', 'सांज',
       'बिपिनसांगळे', 'nimita', 'केशवकूल', 'बेफ़िकीर', 'स्पार्टाकस',
       'सोहनी सोहनी', 'Sujata Siddha', 'मी मधुरा', 'किल्ली', 'जव्हेरगंज',
       'विद्या भुतकर', 'पायस', 'शुभम्', 'सुर्या---', 'SharmilaR'],
      dtype='object', name='author')

In [16]:

#Shuffle the DataFrame
shuffled_df = top_authors_df.sample(frac=1, random_state=42).reset_index(drop=True)

shuffled_df


,title,link,subtitle,author,content,clean_content
0,वारसा,https://www.maayboli.com/node/78299,लेखनाचा धागा Sep 9 2021 - 8:12am,सांज,तो अवघड घाट उतरताना नेहाच्या मनात थोडीशी धाकध...,तो अवघड घाट उतरताना नेहाच्या मनात थोडीशी धाकधू...
1,नोट,https://www.maayboli.com/node/85367,लेखनाचा धागा Aug 31 2024 - 12:11pm,बिपिनसांगळे,नोट\nरात्र झालेली . बाहेर पावसाला उजाडलेलं .\n...,नोट रात्र झालेली . बाहेर पावसाला उजाडलेलं . अं...
2,मला काहीच आठवत नाहीये भाग ३ - अनोळखी ओढ...,https://www.maayboli.com/node/68477,लेखनाचा धागा Dec 30 2018 - 4:45am,अज्ञातवासी,भाग १\nhttps://www.maayboli.com/node/68392\nभा...,"भाग १ भाग २ ""मने, उठतेय ना?"" मला ओरडून सांगावं..."
3,©अज्ञातवासी - भाग ७ - नायक!,https://www.maayboli.com/node/77171,लेखनाचा धागा Nov 5 2020 - 12:46pm,अज्ञातवासी,©सर्व हक्क लेखकास्वाधीन. कुठल्याही प्रकारे पुन...,©सर्व हक्क लेखकास्वाधीन. कुठल्याही प्रकारे पुन...
4,युगांतर - आरंभ अंताचा! भाग-२,https://www.maayboli.com/node/70709,लेखनाचा धागा Jul 20 2019 - 7:39am,मी मधुरा,वशिष्ठ मुनींच्या आश्रमात एकच गोंधळ माजला होता....,वशिष्ठ मुनींच्या आश्रमात एकच गोंधळ माजला होता....
...,...,...,...,...,...,...
1523,नातीगोती- भाग १,https://www.maayboli.com/node/63469,लेखनाचा धागा Jan 22 2019 - 1:17pm,अज्ञातवासी,"""पप्पा!""\n""बोल!""\n""अंह असा रिप्लाय नाही द्यायच...","""पप्पा!"" ""बोल!"" ""अंह असा रिप्लाय नाही द्यायचा,..."
1524,स्वप्नाळू : भाग ३,https://www.maayboli.com/node/61787,लेखनाचा धागा Mar 4 2017 - 2:43am,विद्या भुतकर,ती रात्र वगळता मुक्ता पुन्हा तिच्या विश्वात रम...,ती रात्र वगळता मुक्ता पुन्हा तिच्या विश्वात रम...
1525,अनंत प्रेमचक्र अर्थात The Infinite loop of lov...,https://www.maayboli.com/node/72022,लेखनाचा धागा Feb 17 2020 - 7:27am,शुभम्,अनंत प्रेमचक्र अर्थात The Infinite loop of lov...,अनंत प्रेमचक्र अर्थात The Infinite loop of lov...
1526,थोडे अद्भुत थोडे गूढ - २,https://www.maayboli.com/node/50513,लेखनाचा धागा Sep 10 2014 - 2:18am,स्पार्टाकस,हेनरी मॉरीसन फ्लॅगर हा एकोणिसाव्या शतकातील एक ...,हेनरी मॉरीसन फ्लॅगर हा एकोणिसाव्या शतकातील एक ...


In [17]:
# Create feature vectors
vectorizer = TfidfVectorizer(
    analyzer='char_wb',  # Character n-grams with word boundaries
    ngram_range=(2, 5),  # Use 2-5 character sequences
    max_features=10000,
    sublinear_tf=True
)

X = vectorizer.fit_transform(shuffled_df['clean_content'])
y = shuffled_df['author']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [18]:

print("Unique authors in y_train:\n",(y_train.unique()))
print("\nUnique authors in y_test:\n", (y_test.unique()))
print(set(y_train.unique()) == set(y_test.unique()))


Unique authors in y_train:
 ['बिपिनसांगळे' 'निमिष_सोनार' 'अज्ञातवासी' 'SharmilaR' 'सोहनी सोहनी'
 'मिरिंडा' 'Sujata Siddha' 'मिलिंद महांगडे' 'स्पार्टाकस' 'सुर्या---'
 'nimita' 'मी मधुरा' 'विद्या भुतकर' 'पायस' 'जव्हेरगंज' 'केशवकूल' 'किल्ली'
 'बेफ़िकीर' 'शुभम्' 'सांज']

Unique authors in y_test:
 ['स्पार्टाकस' 'जव्हेरगंज' 'विद्या भुतकर' 'nimita' 'अज्ञातवासी'
 'बिपिनसांगळे' 'Sujata Siddha' 'पायस' 'शुभम्' 'मी मधुरा' 'बेफ़िकीर'
 'मिरिंडा' 'सोहनी सोहनी' 'सुर्या---' 'मिलिंद महांगडे' 'सांज' 'SharmilaR'
 'निमिष_सोनार' 'केशवकूल' 'किल्ली']
True


In [19]:
model = MultinomialNB()
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

                precision    recall  f1-score   support

     SharmilaR       0.00      0.00      0.00        12
 Sujata Siddha       1.00      0.25      0.40        20
        nimita       0.93      0.82      0.88        17
    अज्ञातवासी       0.25      0.98      0.40        65
        किल्ली       0.00      0.00      0.00        15
       केशवकूल       1.00      0.25      0.40        24
     जव्हेरगंज       0.00      0.00      0.00        17
   निमिष_सोनार       0.82      0.74      0.78        31
          पायस       1.00      0.53      0.69        17
   बिपिनसांगळे       1.00      0.32      0.49        31
       बेफ़िकीर       1.00      0.82      0.90        28
       मिरिंडा       1.00      0.80      0.89        30
मिलिंद महांगडे       0.85      1.00      0.92        28
      मी मधुरा       0.00      0.00      0.00        18
  विद्या भुतकर       1.00      0.05      0.10        20
         शुभम्       1.00      0.50      0.67        12
          सांज       0.96      0.89      0.92  

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [20]:

from sklearn.metrics import accuracy_score

for author in np.unique(y_test):
    author_indices = np.where(y_test == author)[0]
    author_y_test = y_test.iloc[author_indices]
    author_y_pred = y_pred[author_indices]

    accuracy = accuracy_score(author_y_test, author_y_pred)
    print(f"Accuracy for author '{author}': {accuracy:.4f}")


Accuracy for author 'SharmilaR': 0.0000
Accuracy for author 'Sujata Siddha': 0.2500
Accuracy for author 'nimita': 0.8235
Accuracy for author 'अज्ञातवासी': 0.9846
Accuracy for author 'किल्ली': 0.0000
Accuracy for author 'केशवकूल': 0.2500
Accuracy for author 'जव्हेरगंज': 0.0000
Accuracy for author 'निमिष_सोनार': 0.7419
Accuracy for author 'पायस': 0.5294
Accuracy for author 'बिपिनसांगळे': 0.3226
Accuracy for author 'बेफ़िकीर': 0.8214
Accuracy for author 'मिरिंडा': 0.8000
Accuracy for author 'मिलिंद महांगडे': 1.0000
Accuracy for author 'मी मधुरा': 0.0000
Accuracy for author 'विद्या भुतकर': 0.0500
Accuracy for author 'शुभम्': 0.5000
Accuracy for author 'सांज': 0.8889
Accuracy for author 'सुर्या---': 0.0000
Accuracy for author 'सोहनी सोहनी': 0.0000
Accuracy for author 'स्पार्टाकस': 1.0000
